In [176]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Colab/Practicum/HACKATON/df_new.csv")
data_ml = data[["idade","genero","estado_civil", "faixa_renda", "bloco"]]

In [180]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1645 entries, 0 to 1644
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0.1                1645 non-null   int64  
 1   Unnamed: 0                  1645 non-null   int64  
 2   morador                     1645 non-null   object 
 3   numero_controle             1645 non-null   int64  
 4   bloco                       1645 non-null   object 
 5   bloco_classificacao         1645 non-null   object 
 6   dt_visita                   1645 non-null   object 
 7   dt_termino                  1645 non-null   object 
 8   tempo_total_entrev          1645 non-null   object 
 9   idade                       1645 non-null   int64  
 10  pais                        1645 non-null   object 
 11  estado                      1640 non-null   object 
 12  cidade                      1640 non-null   object 
 13  bairro_bh                   1135 

## Clustering

In [209]:
from sklearn.cluster import KMeans

df = pd.read_csv("/content/drive/MyDrive/Colab/Practicum/HACKATON/df_new.csv") 

# Select features
features = ['idade', 'genero', 'estado_civil']

# Convert categorical features to numerical
df_numerical = pd.get_dummies(df[features])

# Cluster the data
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(df_numerical)

# Recommend a block for a chosen person
chosen_person = df[(df['idade'] == 30) & (df['genero'] == 'Feminino') & (df['estado_civil'] == 'Solteiro(a)')]
if chosen_person.empty:
    recommended_block = df['bloco'].sample().values[0]
else:
    chosen_cluster = chosen_person['cluster'].values[0]
    recommended_block = df[df['cluster'] == chosen_cluster]['bloco'].mode().values[0]
print(f"The recommended block for the chosen person is {recommended_block}.")


The recommended block for the chosen person is Baianas Ozadas.


# ML

In [204]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer, precision_score

## DecisionTreeClassifier

In [ ]:
# Load the data
data = pd.read_csv("/content/drive/MyDrive/Colab/Practicum/HACKATON/df_new.csv")
data = data[["idade","genero","estado_civil", "faixa_renda", "bloco"]]

# Define the features and target
features = ['idade', 'genero', 'estado_civil', 'faixa_renda']
target = 'bloco'

# Convert categorical features into numeric features
data = pd.get_dummies(data, columns=features)

# Split the data into features and target variables
X = data.drop(target, axis=1)
y = data[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
clf = DecisionTreeClassifier(random_state=42)

# Fit the model to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.22188449848024316


In [ ]:
# Define the hyperparameters to tune
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the scoring metric for cross-validation
scorer = make_scorer(precision_score, average='micro')

# Perform cross-validation with grid search on the training data
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring=scorer)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print('Best hyperparameters:', grid_search.best_params_)

# Train the model with the best hyperparameters on the entire training data
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate the model on the testing data
accuracy = best_model.score(X_test, y_test)
print('Accuracy:', accuracy)

# Evaluate the model using cross-validation on the entire data set
scores = cross_val_score(best_model, X, y, cv=5, scoring=scorer)
print('Precision scores:', scores)
print('Mean precision score:', scores.mean())

Best hyperparameters: {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}
Accuracy: 0.331306990881459
Precision scores: [0.31914894 0.34650456 0.31610942 0.33130699 0.31306991]
Mean precision score: 0.32522796352583583


## RandomForestClassifier

In [169]:
# Load the data
data = pd.read_csv("/content/drive/MyDrive/Colab/Practicum/HACKATON/df_new.csv")
data = data[["idade","genero","estado_civil", "bloco"]]

# Define the features and target
features = ['idade', 'genero', 'estado_civil', 'faixa_renda']
target = 'bloco'

# Convert categorical features into numeric features
data = pd.get_dummies(data, columns=['genero', 'estado_civil'])

# Split the data into features and target variables
X = data.drop(target, axis=1)
y = data[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop(target, axis=1), data[target], test_size=0.2, random_state=42)

# Define the model
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

# Fit the model to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.3465045592705167


In [171]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1645 entries, 0 to 1644
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   idade                        1645 non-null   int64 
 1   bloco                        1645 non-null   object
 2   genero_Feminino              1645 non-null   uint8 
 3   genero_Masculino             1645 non-null   uint8 
 4   estado_civil_Casado (a)      1645 non-null   uint8 
 5   estado_civil_Divorciado (a)  1645 non-null   uint8 
 6   estado_civil_Nao respondeu   1645 non-null   uint8 
 7   estado_civil_Outro           1645 non-null   uint8 
 8   estado_civil_Solteiro (a)    1645 non-null   uint8 
 9   estado_civil_Viuvo (a)       1645 non-null   uint8 
dtypes: int64(1), object(1), uint8(8)
memory usage: 38.7+ KB


### Custom Test

In [ ]:
# Define the allowed options for the estado_civil column
allowed_estado_civil = ['Solteiro (a)', 'Casado (a)', 'Outro', 'Divorciado (a)', 'Viuvo (a)', 'Nao respondeu']
# Generate a list of enumerated options for the estado_civil column
enumerated_estado_civil = list(enumerate(allowed_estado_civil, start=1))
# Convert the enumerated options to a list of strings
estado_civil_options = [f'{index}. {option}' for index, option in enumerated_estado_civil]

# Define the allowed options for the faixa_renda column
allowed_faixa_renda = [
    'Ate um salario minimo (R$ 954,00 )',
    'de 1 a 2 salarios (R$ 954,00 – R$ 1908,00)',
    'de 2 a 3 salarios (R$ 1875,00 – R$ 2862,00)',
    'de 3 a 4 salarios (R$ 2812,00 – R$ 3816,00)',
    'de 4 a 5 salarios (R$ 3749,00 – R$ 4770,00)',
    'de 5 a 6 salarios (R$ 4686,00 – R$ 5724,00)',
    'de 6 a 7 salarios (R$ 5623,00 – R$ 6678,00)',
    'de 7 a 8 salarios (R$ 6560,00 – R$ 7632,00)',
    'de 8 a 9 salarios (R$ 7497,00 – R$ 8586,00)',
    'de 9 a 10 salarios (R$ 8434,00 – R$ 9540,00)',
    'Acima de 10 salarios (acima de R$ 9540,00)',
    'Nao sabe',
    'Nao respondeu'
]

# Generate a list of enumerated options for the faixa_renda column
enumerated_faixa_renda = list(enumerate(allowed_faixa_renda, start=1))
# Convert the enumerated options to a list of strings
faixa_renda_options = [f'{index}. {option}' for index, option in enumerated_faixa_renda]

# Prompt the user for input and show the enumerated options for the estado_civil column
while True:
    idade = int(input('Qual a sua idade? '))
    genero = input('Qual o seu gênero (Feminino/Masculino)? ')
    estado_civil = input('Qual é o seu estado civil?\n' + '\n'.join(estado_civil_options) + '\n')
    print('Qual é a sua faixa de renda mensal (em reais)?')
    print('\n'.join(faixa_renda_options))
    faixa_renda = input()

    # Parse the user's input to get the corresponding allowed option
    try:
        faixa_renda_idx = int(faixa_renda)
        faixa_renda = enumerated_faixa_renda[faixa_renda_idx - 1][1]
    except (ValueError, IndexError):
        print(f'Opção inválida para a faixa de renda. Por favor, tente novamente.')
        continue

    # Convert the user input to the same categorical format as the original data
    genero_cat = 'Feminino' if genero.lower() == 'feminino' else 'Masculino'
    estado_civil_cat = estado_civil
    faixa_renda_cat = faixa_renda

    # Create a new DataFrame with the user input
    custom_row = pd.DataFrame({
        'idade': [idade],
        'genero': [genero_cat],
        'estado_civil': [estado_civil_cat],
        'faixa_renda': [faixa_renda_cat]
    })

    # Convert the categorical variables into numerical variables using the same mapping as before
    custom_row = pd.get_dummies(custom_row, columns=['genero', 'estado_civil', 'faixa_renda'])

    # Reorder the columns to match the order of the original data
    custom_row = custom_row.reindex(columns=X.columns, fill_value=0)

    # Make predictions for the custom row
    y_pred_custom = clf.predict(custom_row)

    # Print the predicted target variable
    print(y_pred_custom)

    break

Qual a sua idade? 50
Qual o seu gênero (Feminino/Masculino)? feminino
Qual é o seu estado civil?
1. Solteiro (a)
2. Casado (a)
3. Outro
4. Divorciado (a)
5. Viuvo (a)
6. Nao respondeu
4
Qual é a sua faixa de renda mensal (em reais)?
1. Ate um salario minimo (R$ 954,00 )
2. de 1 a 2 salarios (R$ 954,00 – R$ 1908,00)
3. de 2 a 3 salarios (R$ 1875,00 – R$ 2862,00)
4. de 3 a 4 salarios (R$ 2812,00 – R$ 3816,00)
5. de 4 a 5 salarios (R$ 3749,00 – R$ 4770,00)
6. de 5 a 6 salarios (R$ 4686,00 – R$ 5724,00)
7. de 6 a 7 salarios (R$ 5623,00 – R$ 6678,00)
8. de 7 a 8 salarios (R$ 6560,00 – R$ 7632,00)
9. de 8 a 9 salarios (R$ 7497,00 – R$ 8586,00)
10. de 9 a 10 salarios (R$ 8434,00 – R$ 9540,00)
11. Acima de 10 salarios (acima de R$ 9540,00)
12. Nao sabe
13. Nao respondeu
2
['Baianas Ozadas']


In [ ]:
# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the scoring metric for cross-validation
scorer = make_scorer(precision_score, average='micro')

# Perform cross-validation with grid search on the training data
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring=scorer)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print('Best hyperparameters:', grid_search.best_params_)

# Train the model with the best hyperparameters on the entire training data
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate the model on the testing data
accuracy = best_model.score(X_test, y_test)
print('Accuracy:', accuracy)

# Evaluate the model using cross-validation on the entire data set
scores = cross_val_score(best_model, X, y, cv=5, scoring=scorer)
print('Precision scores:', scores)
print('Mean precision score:', scores.mean())

Best hyperparameters: {'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy: 0.3434650455927052
Precision scores: [0.3343465  0.32826748 0.34042553 0.3343465  0.33130699]
Mean precision score: 0.3337386018237082


## SVM

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv("/content/drive/MyDrive/Colab/Practicum/HACKATON/df_new.csv")
data = data[["idade","genero","estado_civil", "faixa_renda", "bloco"]]

# Define the features and target
features = ['idade', 'genero', 'estado_civil', 'faixa_renda']
target = 'bloco'

# Split the data into features and target variables
X = data.drop(target, axis=1)
y = data[target]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the column transformer for one-hot encoding categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['genero', 'estado_civil']),
    ])

# Define the SVM classifier
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC(kernel='linear', random_state=42))])

# Fit the model on the training data
clf.fit(X_train, y_train)

# Test the model on the testing data
score = clf.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.3282674772036474


## Naive Bayes

In [161]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import csr_matrix
import pandas as pd

# Define the features and target
features = ['idade', 'genero', 'estado_civil', 'faixa_renda']
target = 'bloco'

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Define the column transformer for one-hot encoding categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['genero', 'estado_civil']),
    ])

# Preprocess the data and convert to sparse matrix format
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
X_train_sparse = csr_matrix(X_train)
X_test_sparse = csr_matrix(X_test)

# Define the Naive Bayes classifier
NaiveBayes = MultinomialNB()

# Fit the model on the training data
NaiveBayes.fit(X_train_sparse, y_train)

# Test the model on the testing data
score = NaiveBayes.score(X_test_sparse, y_test)

print("Accuracy:", score)


Accuracy: 0.3586626139817629


## KNN

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Converter variáveis categóricas em numéricas
le = LabelEncoder()
data_ml["genero"] = le.fit_transform(data_ml["genero"])
data_ml["estado_civil"] = le.fit_transform(data_ml["estado_civil"])
data_ml["faixa_renda"] = le.fit_transform(data_ml["faixa_renda"])
data_ml["bloco"] = le.fit_transform(data_ml["bloco"])

# Dividir os dados em conjunto de treinamento e conjunto de teste
X = data_ml.drop("bloco", axis=1)
y = data_ml["bloco"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo KNN com k=5
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = knn.predict(X_test)

# Avaliar a acurácia do modelo
score = accuracy_score(y_test, y_pred)
print("Accuracy:", score)

Accuracy: 0.2948328267477204


# # Python Script

In [156]:
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, make_scorer, precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix

def run_ml_models(data, features, target):
    results = []

    # Convert categorical features into numeric features
    le = LabelEncoder()
    for feature in features:
        if data[feature].dtype == "object":
            data[feature] = le.fit_transform(data[feature])

    # Split the data into features and target variables
    X = data.drop(target, axis=1)
    y = data[target]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Models and names
    models = [
        ('DecisionTree', DecisionTreeClassifier(random_state=42)),
        ('RandomForest', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)),
        ('SVM', Pipeline(steps=[('preprocessor', ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), features)])), ('classifier', SVC(kernel='linear', random_state=42))])),
        ('NaiveBayes', MultinomialNB()),
        ('KNN', KNeighborsClassifier(n_neighbors=5))
    ]

    for name, model in models:
        start_time = time.time()
        
        if name == 'NaiveBayes':
            preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), features)])
            X_train_transformed = csr_matrix(preprocessor.fit_transform(X_train))
            X_test_transformed = csr_matrix(preprocessor.transform(X_test))
            model.fit(X_train_transformed, y_train)
            y_pred = model.predict(X_test_transformed)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
        end_time = time.time()
        processing_time = end_time - start_time
        accuracy = accuracy_score(y_test, y_pred)
        results.append([name, processing_time, accuracy])

    results_df = pd.DataFrame(results, columns=['Model', 'Processing Time', 'Accuracy'])
    return results_df

# Load the data
data = pd.read_csv("/content/drive/MyDrive/Colab/Practicum/HACKATON/df_new.csv")
data = data[["idade", "genero", "estado_civil", "faixa_renda", "bloco"]]

# Define the features and target
features = ['idade', 'genero', 'estado_civil', 'faixa_renda']
target = 'bloco'

# Run the ML models
results_df = run_ml_models(data, features, target)
print(results_df.sort_values(by='Accuracy', ascending=False))

          Model  Processing Time  Accuracy
1  RandomForest         0.181539  0.352584
3    NaiveBayes         0.010923  0.328267
2           SVM         0.175726  0.325228
4           KNN         0.012937  0.294833
0  DecisionTree         0.006779  0.206687


In [157]:
# loading library
import pickle

In [164]:
import pickle

filename = "my_model.pickle"

# save model
pickle.dump(NaiveBayes, open(filename, "wb"))